# NTK classifier for Cora

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk_certify import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        n_per_class = 20,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

certificate_params = dict(
    n_adversarial = 10, # number adversarial nodes
    perturbation_model = "l0",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [5]:
def run_exp(n_seeds, data_params, model_params, certificate_params,
            verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, certificate_params,
                  verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [36]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "citeseer"
other_params["device"] = 0
certificate_params["n_adversarial"] = 10
certificate_params["delta"] = 0.01
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

{'accuracy': 0.7441860437393188,
 'accuracy_ub': 0.5627906918525696,
 'accuracy_lb': 0.6930232644081116,
 'accuracy_cert': 0.6232557892799377,
 'min_ypred': -1.6106116771697998,
 'max_ypred': 1.5041157007217407,
 'min_ylb': -1.814160704612732,
 'max_ylb': 1.6415013074874878,
 'min_yub': -7.790926456451416,
 'max_yub': 9.772697448730469,
 'min_ntklb': 0.2097131890176563,
 'max_ntklb': 53.1176356162013,
 'min_ntkub': 2.464936557120615,
 'max_ntkub': 258.0180363996275,
 'min_ntklabeled': 2.3843084012159235,
 'max_ntklabeled': 78.09282218277724,
 'min_ntkunlabeled': 2.1946010562937928,
 'max_ntkunlabeled': 28.78762262343896,
 'cond': 14493.727539407795}

In [20]:
a = torch.Tensor(([1, 2, 1, 1], [2, 2, 3, 2], [4, 3, 3, 3], [4,4,4,5]))
a_ub = torch.Tensor(([1, 2, 3, 1], [2, 2, 3, 2], [4, 5, 3, 3], [4,6,6,5]))
pred_orig = a.argmax(1)
pred_orig_lb = a[range(4), pred_orig]
mask = torch.ones(a.shape, dtype=torch.bool)
mask[range(4), pred_orig] = False
pred_other_ub = a_ub[mask].reshape((a.shape[0], a.shape[1]-1))
print(pred_orig_lb)
print(pred_other_ub)
print(pred_orig_lb.reshape(-1, 1))
print(pred_other_ub > pred_orig_lb.reshape(-1, 1))
mask = (pred_other_ub > pred_orig_lb.reshape(-1, 1)).sum(dim=1)
print((mask == 0).sum())


tensor([2., 3., 4., 5.])
tensor([[1., 3., 1.],
        [2., 2., 2.],
        [5., 3., 3.],
        [4., 6., 6.]])
tensor([[2.],
        [3.],
        [4.],
        [5.]])
tensor([[False,  True, False],
        [False, False, False],
        [ True, False, False],
        [False,  True,  True]])
tensor(1)


# SVM

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 1
data_params["dataset"] = "cora"
model_params["pred_method"] = "krr"
n_seeds = 2
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)